BroadCast Variables
====================

In [23]:
# Un hash con los productos y sus nombres, hacemos un broadcast
productNames = {1:'papas',2:'cebollas',3:'tomates',4:'zanahorias',5:'batatas',6:'peras',7:'cilantro',8:'apio',9:'morrones',10:'manzanas',11:'naranjas'}
bproductNames = sc.broadcast(productNames)

# Vamos a suponer que tenemos un RDD de productos por sus IDs identificando ventas de los mismos
prodsList = [1,11,1,4,5,11,2,3,4,5,6,4,5,4,3,2,1,11,2,3,4,5,6,4,3,2,1,1]
prods = sc.parallelize(prodsList,3)

# Buscamos los productos que se vendieron mas de 4 veces
popularProds = prods.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>=4)

# Buscamos los nombres
popularProds = popularProds.map(lambda x:(bproductNames.value[x[0]],x[0],x[1]))
print popularProds.collect()                

[('tomates', 3, 4), ('papas', 1, 5), ('zanahorias', 4, 6), ('cebollas', 2, 4), ('batatas', 5, 4)]


SON
===

Version paralelizable de A-Priori


In [33]:
import os.path
import re,string
import itertools

def filterDictionary(d,support):
    for item in d.keys():
        if d[item]<support:
            del(d[item])
    return d

# N-pass in memory apriori algorithm 
def apriori(listOfBaskets):
    listOfBaskets=list(listOfBaskets)
    # This is your parameter
    minsupport = 3
    frequentItems = dict()
    result = dict()
    # First scan all items
    for basket in listOfBaskets:       
        for item in sorted(basket):
            if (item,) in frequentItems:
                frequentItems[(item,)]+=1
            else:
                frequentItems[(item,)]=1
    # Filter items mantain only individual items with min support
    frequentItems = filterDictionary(frequentItems,minsupport)
    order = 2
    result.update(frequentItems)   
    while(frequentItems):
        newFrequentItems = dict()
        for basket in listOfBaskets:
            all_combinations = itertools.combinations(sorted(basket),order)
            for combination in all_combinations:
                subcombination = itertools.combinations(list(combination),order-1)
                # All items must be frequent
                isfrequent = True
                for item in sorted(subcombination):
                    if item not in frequentItems:
                        isfrequent = False
                        break
                # if all subcombinations are freuqnt then add combination
                if isfrequent:
                    if tuple(sorted(combination)) in newFrequentItems:
                        newFrequentItems[tuple(sorted(combination))]+=1
                    else:
                        newFrequentItems[tuple(sorted(combination))]=1
        # Now filter each member of the dictionary by support       
        newFrequentItems = filterDictionary(newFrequentItems,minsupport)
        frequentItems = newFrequentItems       
        result.update(frequentItems)
        order = order+1
    vector = []
    for item in result.keys():
        vector.append((item,1))
    return vector

def filterByList(listaFreqs):
    def bar(row):
        ret = []
        # We have to check if the items in listaFreqs are in this row
        for item in listaFreqs:
            if set(item).issubset(set(row)):
                ret.append((item,1))
        return ret
    return bar


fileName = 'data/apriori/datosapriori.txt'
itemsRDD = (sc.textFile(fileName, 4))
print itemsRDD.count()
print itemsRDD.first()
print "-----------------"
itemsRDD = itemsRDD.map(lambda x:re.sub(r'[^a-zA-Z0-9 ]', '', x)  ).map(lambda x:x.split())
print itemsRDD.first()
print "-----------------"
itemsRDD1 = itemsRDD.mapPartitions(apriori).reduceByKey(lambda x,y:x).map(lambda x:x[0])
print itemsRDD1.first()
print "-----------------"
# Now we have to map again
listaItemsFrecuentes = itemsRDD1.collect()

print listaItemsFrecuentes
print "================="
print itemsRDD.flatMap(filterByList(listaItemsFrecuentes)).reduceByKey(lambda x,y:x+y).filter(lambda x:x[1]>12).collect()

64
tomate, pepsi
-----------------
[u'tomate', u'pepsi']
-----------------
(u'aceitunas', u'queso')
-----------------
[(u'aceitunas', u'queso'), (u'aceitunas', u'papas', u'salame'), (u'carne', u'coca'), (u'carne', u'papas'), (u'papas', u'queso'), (u'ajo',), (u'queso',), (u'huevos',), (u'carne',), (u'queso', u'salame'), (u'leche',), (u'paales',), (u'cerveza',), (u'salame',), (u'aceitunas', u'queso', u'salame'), (u'papas', u'queso', u'salame'), (u'coca', u'papas'), (u'aceitunas', u'papas'), (u'cerveza', u'paales'), (u'coca',), (u'aceitunas', u'papas', u'queso', u'salame'), (u'aceitunas',), (u'pan',), (u'tomate',), (u'lechuga',), (u'carne', u'coca', u'papas'), (u'aceitunas', u'salame'), (u'aceitunas', u'papas', u'queso'), (u'papas',), (u'papas', u'salame')]
[((u'huevos',), 14), ((u'queso',), 15), ((u'coca',), 15), ((u'tomate',), 13), ((u'papas',), 28)]


Calculando Pi en Spark
-----------------------

Because we can

In [29]:
from random import random
NUM_SAMPLES = 10000000
def sample(p):
    x, y = random(), random()
    return 1 if x*x + y*y < 1 else 0

count = sc.parallelize(xrange(0, NUM_SAMPLES)).map(sample) \
             .reduce(lambda a, b: a + b)
print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)

Pi is roughly 3.141625


The Monty Hall Problem
----------------------

In [31]:
from random import randrange
NUM_SAMPLES = 100000
def sample(p):
    prize_door = randrange(3)+1
    human_door = randrange(3)+1
    if prize_door != human_door:
        return 1
    else:
        return 0
count = sc.parallelize(xrange(0, NUM_SAMPLES)).map(sample) \
             .reduce(lambda a, b: a + b)
print "Switching wins in %f" % (float(count) / NUM_SAMPLES * 100)    + " % of cases"

Switching wins in 66.428000 % of cases
